# Human Activity Recognition

# Prediction Assignment Writeup

## Background

Using devices such as Jawbone Up, Nike FuelBand, and Fitbit it is now possible to collect a large amount of data about personal activity relatively inexpensively. These type of devices are part of the quantified self movement – a group of enthusiasts who take measurements about themselves regularly to improve their health, to find patterns in their behavior, or because they are tech geeks. One thing that people regularly do is quantify how much of a particular activity they do, but they rarely quantify how well they do it. In this project, your goal will be to use data from accelerometers on the belt, forearm, arm, and dumbell of 6 participants. They were asked to perform barbell lifts correctly and incorrectly in 5 different ways. More information is available from the website here: http://web.archive.org/web/20161224072740/http:/groupware.les.inf.puc-rio.br/har (see the section on the Weight Lifting Exercise Dataset).

# Goal
The goal of your project is to predict the manner in which they did the exercise. This is the "classe" variable in the training set. You may use any of the other variables to predict with. You should create a report describing how you built your model, how you used cross validation, what you think the expected out of sample error is, and why you made the choices you did. You will also use your prediction model to predict 20 different test cases.

# Intall Packages

In [1]:
options(warn=-1)

In [2]:
install.packages('caret')
install.packages('rpart')
install.packages('rpart.plot')
install.packages("ipred")
install.packages("randomForest")

package 'caret' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\PAVILION 2\AppData\Local\Temp\RtmpCgPVBS\downloaded_packages
package 'rpart' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\PAVILION 2\AppData\Local\Temp\RtmpCgPVBS\downloaded_packages
package 'rpart.plot' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\PAVILION 2\AppData\Local\Temp\RtmpCgPVBS\downloaded_packages
package 'ipred' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\PAVILION 2\AppData\Local\Temp\RtmpCgPVBS\downloaded_packages
package 'randomForest' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\PAVILION 2\AppData\Local\Temp\RtmpCgPVBS\downloaded_packages


In [5]:
library(caret)
library(rpart)
library(rpart.plot)
library(ipred)
library(randomForest)

# Loading th data

In [20]:
training  <- read.csv('https://d396qusza40orc.cloudfront.net/predmachlearn/pml-training.csv')
testing <- read.csv('https://d396qusza40orc.cloudfront.net/predmachlearn/pml-testing.csv')

In [21]:
head(training,3)

,X,user_name,raw_timestamp_part_1,raw_timestamp_part_2,cvtd_timestamp,new_window,num_window,roll_belt,pitch_belt,yaw_belt,...,gyros_forearm_x,gyros_forearm_y,gyros_forearm_z,accel_forearm_x,accel_forearm_y,accel_forearm_z,magnet_forearm_x,magnet_forearm_y,magnet_forearm_z,classe
,<int>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<fct>
1,1,carlitos,1323084231,788290,05/12/2011 11:23,no,11,1.41,8.07,-94.4,...,0.03,0.00,-0.02,192,203,-215,-17,654,476,A
2,2,carlitos,1323084231,808298,05/12/2011 11:23,no,11,1.41,8.07,-94.4,...,0.02,0.00,-0.02,192,203,-216,-18,661,473,A
3,3,carlitos,1323084231,820366,05/12/2011 11:23,no,11,1.42,8.07,-94.4,...,0.03,-0.02,0.00,196,204,-213,-18,658,469,A


In [22]:
dim(training)

[1] 19622   160

In [23]:
head(testing,3)

,X,user_name,raw_timestamp_part_1,raw_timestamp_part_2,cvtd_timestamp,new_window,num_window,roll_belt,pitch_belt,yaw_belt,...,gyros_forearm_x,gyros_forearm_y,gyros_forearm_z,accel_forearm_x,accel_forearm_y,accel_forearm_z,magnet_forearm_x,magnet_forearm_y,magnet_forearm_z,problem_id
,<int>,<fct>,<int>,<int>,<fct>,<fct>,<int>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,pedro,1323095002,868349,05/12/2011 14:23,no,74,123.00,27.00,-4.75,...,0.74,-3.34,-0.59,-110,267,-149,-714,419,617,1
2,2,jeremy,1322673067,778725,30/11/2011 17:11,no,431,1.02,4.87,-88.90,...,1.12,-2.78,-0.18,212,297,-118,-237,791,873,2
3,3,jeremy,1322673075,342967,30/11/2011 17:11,no,439,0.87,1.82,-88.50,...,0.18,-0.79,0.28,154,271,-129,-51,698,783,3


In [24]:
dim(testing)

[1]  20 160

# Cleaning the data

### Cleaning the training data

In [25]:
# remove variables with nearly zero variance
nzv <- nearZeroVar(training)
training <- training[, -nzv]

# remove variables that are almost always NA
remove_NA <- sapply(training, function(x) mean(is.na(x))) > 0.95
training <- training[, remove_NA==F]

# remove first 5 useless variables 
training <- training[, -(1:5)]

In [27]:
dim(training)

[1] 19622    54

### Cleaning the testing data

In [28]:
# remove variables with nearly zero variance
nzv <- nearZeroVar(testing)
testing <- testing[, -nzv]

# remove variables that are almost always NA
remove_NA <- sapply(testing, function(x) mean(is.na(x))) > 0.95
testing <- testing[, remove_NA==F]

# remove first 5 useless variables 
testing <- testing[, -(1:5)]

In [29]:
dim(testing)

[1] 20 54

# Partitioning Training Data

In [30]:
inTrain <- createDataPartition(training$classe, p=0.7, list=FALSE)
myTraining <- training[inTrain, ]
myTesting <- training[-inTrain, ]

In [31]:
dim(myTraining)
dim(myTesting)

[1] 13737    54

[1] 5885   54

# Creating the Models

## Tree Model

In [32]:
# Create the tree model
dt <- rpart(formula = classe ~ ., 
                        data = myTraining, 
                        method = "class")

In [33]:
# Generate predicted classes using the dt model
dt_prediction <- predict(object = dt,  
                          newdata = myTesting,   
                          type = "class",
                          parms = list(split = "gini"))  
# Calculate the confusion matrix for the test set
confusionMatrix(data =dt_prediction ,       
                  reference = myTesting$classe) 

Confusion Matrix and Statistics

          Reference
Prediction    A    B    C    D    E
         A 1490  277   38  111   62
         B   37  608   65   35   86
         C   21   84  835   78   56
         D   96  134   66  675  147
         E   30   36   22   65  731

Overall Statistics
                                          
               Accuracy : 0.7373          
                 95% CI : (0.7259, 0.7485)
    No Information Rate : 0.2845          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.666           
                                          
 Mcnemar's Test P-Value : < 2.2e-16       

Statistics by Class:

                     Class: A Class: B Class: C Class: D Class: E
Sensitivity            0.8901   0.5338   0.8138   0.7002   0.6756
Specificity            0.8841   0.9530   0.9508   0.9100   0.9681
Pos Pred Value         0.7533   0.7316   0.7775   0.6038   0.8269
Neg Pred Value         0.9529   0.894

In [34]:
# Create the Bagging model
b_model <- bagging(formula = classe ~ ., 
                        data = myTraining,
                        coob = TRUE)

# Print the model
print(b_model)


Bagging classification trees with 25 bootstrap replications 

Call: bagging.data.frame(formula = classe ~ ., data = myTraining, coob = TRUE)

Out-of-bag estimate of misclassification error:  0.0059 



In [35]:
# Generate predicted classes using the model object
b_prediction <- predict(object = b_model,    
                              newdata = myTesting,  
                            type = "class")  # return classification labels

# Calculate the confusion matrix for the test set
confusionMatrix(data = b_prediction ,       
                  reference = myTesting$classe) 

Confusion Matrix and Statistics

          Reference
Prediction    A    B    C    D    E
         A 1673   13    0    0    0
         B    1 1122    0    0    0
         C    0    3 1026    0    0
         D    0    1    0  962    1
         E    0    0    0    2 1081

Overall Statistics
                                          
               Accuracy : 0.9964          
                 95% CI : (0.9946, 0.9978)
    No Information Rate : 0.2845          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.9955          
                                          
 Mcnemar's Test P-Value : NA              

Statistics by Class:

                     Class: A Class: B Class: C Class: D Class: E
Sensitivity            0.9994   0.9851   1.0000   0.9979   0.9991
Specificity            0.9969   0.9998   0.9994   0.9996   0.9996
Pos Pred Value         0.9923   0.9991   0.9971   0.9979   0.9982
Neg Pred Value         0.9998   0.996

In [36]:
#Create the random forest model
rf_model <- randomForest(formula = classe ~ ., 
                               data = myTraining)
# Print the model output
print(rf_model)


Call:
 randomForest(formula = classe ~ ., data = myTraining) 
               Type of random forest: classification
                     Number of trees: 500
No. of variables tried at each split: 7

        OOB estimate of  error rate: 0.2%
Confusion matrix:
     A    B    C    D    E  class.error
A 3905    0    0    0    1 0.0002560164
B    4 2654    0    0    0 0.0015048909
C    0    6 2390    0    0 0.0025041736
D    0    0   12 2240    0 0.0053285968
E    0    0    0    4 2521 0.0015841584


In [37]:
# Generate predicted classes using the random forest
rf_prediction <- predict(object = rf_model,   # model object 
                              newdata = myTesting,  # test dataset
                              type = "class") # return classification labels
                            
# Calculate the confusion matrix for the random forest
confusionMatrix(data =rf_prediction,       # predicted classes
                        reference = myTesting$classe)  # actual classes

Confusion Matrix and Statistics

          Reference
Prediction    A    B    C    D    E
         A 1674    5    0    0    0
         B    0 1134    0    0    0
         C    0    0 1026    5    0
         D    0    0    0  958    2
         E    0    0    0    1 1080

Overall Statistics
                                          
               Accuracy : 0.9978          
                 95% CI : (0.9962, 0.9988)
    No Information Rate : 0.2845          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.9972          
                                          
 Mcnemar's Test P-Value : NA              

Statistics by Class:

                     Class: A Class: B Class: C Class: D Class: E
Sensitivity            1.0000   0.9956   1.0000   0.9938   0.9982
Specificity            0.9988   1.0000   0.9990   0.9996   0.9998
Pos Pred Value         0.9970   1.0000   0.9952   0.9979   0.9991
Neg Pred Value         1.0000   0.998

# Accuracy:
### Tree Model: 0.7373 
### Bagging Model: 0.9964 
### Random Forest Model: 0.9978 

In [40]:
# Random Forest Model yields very good results!

# Predicting on the Testing Data

## Prediction on the Bagging model

In [41]:
# Generate predicted classes using the bagging
b_prediction <- predict(object = b_model,    
                              newdata = testing,  
                            type = "class")  # return classification labels
b_prediction

[1] B A B A A E D B A A B C B A E E A B B B
Levels: A B C D E

# Prediction on the random forest model

In [42]:
# Generate predicted classes using the model object
rf_prediction <- predict(object = rf_model,   # model object 
                              newdata = testing,  # test dataset
                              type = "class") # return classification labels
rf_prediction

1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 
 B  A  B  A  A  E  D  B  A  A  B  C  B  A  E  E  A  B  B  B 
Levels: A B C D E